In [2]:
import requests 
from bs4 import BeautifulSoup 

   
URL = "https://www.livemint.com/market/market-stats/stocks-adani-enterprises-share-price-nse-bse-s0003094" 
r = requests.get(URL) 
   
soup = BeautifulSoup(r.content, 'html5lib') 
   
quotes=[]  # a list to store quotes 
   
div_income_bse = soup.find('div', attrs = {'id':'incomebse'})  


In [3]:
annual_income_table = div_income_bse.find("table")
print(annual_income_table)

<table><tbody><tr><td><strong>Period</strong></td><td><strong>Mar 2024</strong></td><td><strong>Mar 2023</strong></td><td><strong>Mar 2022</strong></td><td><strong>Mar 2021</strong></td></tr><tr><td>Total Revenue </td><td>96,420.98</td><td>1,36,977.76</td><td>69,420.18</td><td>39,537.13</td></tr><tr><td>Selling/ General/ Admin Expenses Total </td><td>2,330.95</td><td>19,666.91</td><td>6,819.42</td><td>4,147.58</td></tr><tr><td>Depreciation/ Amortization </td><td>3,042.15</td><td>2,436.14</td><td>1,247.78</td><td>537.14</td></tr><tr><td>Other Operating Expenses Total </td><td>31,919.42</td><td>292.26</td><td>208.36</td><td>137.71</td></tr><tr><td>Total Operating Expense </td><td>88,311.12</td><td>1,30,295.00</td><td>66,668.13</td><td>37,765.82</td></tr><tr><td>Operating Income </td><td>8,109.86</td><td>6,682.76</td><td>2,752.05</td><td>1,771.31</td></tr><tr><td>Net Income Before Taxes </td><td>4,924.91</td><td>3,249.90</td><td>952.05</td><td>1,085.97</td></tr><tr><td>Net Income </td><td

In [4]:
print(type(annual_income_table))

<class 'bs4.element.Tag'>


In [5]:
tbody = div_income_bse.find("tbody")
heading = tbody.find_all("strong")


In [6]:
print(heading)
print(type(heading))


[<strong>Period</strong>, <strong>Mar 2024</strong>, <strong>Mar 2023</strong>, <strong>Mar 2022</strong>, <strong>Mar 2021</strong>]
<class 'bs4.element.ResultSet'>


In [7]:
new_headings = []
for head in heading:
    test_str = str(head)
    sub1 = "<strong>"
    sub2 = "</strong>"
    idx1 = test_str.find(sub1)
    idx2 = test_str.find(sub2)
    res = test_str[idx1 + len(sub1): idx2]
    new_headings.append(res)
new_headings

['Period', 'Mar 2024', 'Mar 2023', 'Mar 2022', 'Mar 2021']

In [8]:
table_content = div_income_bse.find_all("td")
table_content = table_content[len(new_headings):]

In [9]:
indexList = []
valueDict = {}
temp = 0
for h in range(1,len(new_headings)):
    valueDict[new_headings[h]] = []     #{"":[],"":[],"":[],"":[]}
for i in range (0,len(table_content)):
    test_str = str(table_content[i])
    sub1 = "<td>"
    sub2 = "</td>"
    idx1 = test_str.find(sub1)
    idx2 = test_str.find(sub2)
    res = test_str[idx1 + len(sub1): idx2]
    if i == 0 or i % len(new_headings) == 0:
        indexList.append(res)
    else:
        if(temp == 4):
            temp = 0
        valueDict[list(valueDict.keys())[temp]].append(float(res.replace(',','')))
        temp = temp +1
indexList

['Total Revenue ',
 'Selling/ General/ Admin Expenses Total ',
 'Depreciation/ Amortization ',
 'Other Operating Expenses Total ',
 'Total Operating Expense ',
 'Operating Income ',
 'Net Income Before Taxes ',
 'Net Income ',
 'Diluted Normalized EPS ']

In [10]:
import pandas as pd
df = pd.DataFrame(valueDict)
df.index = indexList
df

,Mar 2024,Mar 2023,Mar 2022,Mar 2021
Total Revenue,96420.98,136977.76,69420.18,39537.13
Selling/ General/ Admin Expenses Total,2330.95,19666.91,6819.42,4147.58
Depreciation/ Amortization,3042.15,2436.14,1247.78,537.14
Other Operating Expenses Total,31919.42,292.26,208.36,137.71
Total Operating Expense,88311.12,130295.00,66668.13,37765.82
Operating Income,8109.86,6682.76,2752.05,1771.31
Net Income Before Taxes,4924.91,3249.90,952.05,1085.97
Net Income,3240.78,2472.94,776.56,922.64
Diluted Normalized EPS,31.25,23.98,7.06,9.99


In [1]:
import requests 
import pandas as pd
from bs4 import BeautifulSoup 
import json
def get_data(URL):
    stockFinancialData = []
    try:
        r = requests.get(URL) 
        soup = BeautifulSoup(r.content, 'html5lib') 
        script = soup.find('script', attrs = {'id':'__NEXT_DATA__'})  
        line_str = str(script)
        first_line = '<script id="__NEXT_DATA__" type="application/json">'
        second_line = "</script>"
        idx1 = line_str.find(first_line)
        idx2 = line_str.find(second_line)
        result = line_str[idx1 + len(first_line): idx2]
        json_object = json.loads(result)
        stockFinancialData = json_object['props']['pageProps']["stockFinancialData"]
        return stockFinancialData
    except Exception as e:
        print(e)
        return stockFinancialData

In [2]:
def data_structuring(stockFinancialData):
    annualCashFlow = {}
    annualBalance = {}
    annualIncome = {}
    quarterlyCashFlow = {}
    quarterlyBalance = {}
    quarterlyIncome = {}
    for i in range(0,len(stockFinancialData)):
        #Checking if the report is Annual or not
        if stockFinancialData[i]["Type"] == "Annual":
            fiscalYear = stockFinancialData[i]["FiscalYear"]
            annualCashFlow[fiscalYear] = {}
            annualBalance[fiscalYear] = {}
            annualIncome[fiscalYear] = {}
            try:
                cashFlow = stockFinancialData[i]["stockFinancialMap"]["CAS"]
                balanceSheet = stockFinancialData[i]["stockFinancialMap"]["BAL"]
                incomeStatement = stockFinancialData[i]["stockFinancialMap"]["INC"]
                for cash in cashFlow:
                    annualCashFlow[fiscalYear][cash["key"]] = cash["value"]
                for balance in balanceSheet:
                    annualBalance[fiscalYear][balance["key"]] = balance["value"]
                for income in incomeStatement:
                    annualIncome[fiscalYear][income["key"]] = income["value"]
            except Exception as e:
                print("Skipping this year for stock",e)
        else:
            endDate = stockFinancialData[i]["EndDate"]
            quarterlyCashFlow[endDate] = {}
            quarterlyBalance[endDate] = {}
            quarterlyIncome[endDate] = {}
            # incomeStatement = []
            try:
                incomeStatement = stockFinancialData[i]["stockFinancialMap"]["INC"]
                cashFlow = stockFinancialData[i]["stockFinancialMap"]["CAS"]
                balanceSheet = stockFinancialData[i]["stockFinancialMap"]["BAL"]
                for cash in cashFlow:
                    quarterlyCashFlow[endDate][cash["key"]] = cash["value"]
                for balance in balanceSheet:
                    quarterlyBalance[endDate][balance["key"]] = balance["value"]
                for income in incomeStatement:
                    quarterlyIncome[endDate][income["key"]] = income["value"]
            except Exception as e:
                # print("Cashflow and balance not found for {}".format(endDate))
                for income in incomeStatement:
                    quarterlyIncome[endDate][income["key"]] = income["value"]
    return annualCashFlow,annualBalance,annualIncome,quarterlyCashFlow,quarterlyBalance,quarterlyIncome
            



In [3]:
characters_to_replace = ['/', '(', ')', '-']
def change_column_names(s, chars, replacement='_'):
    for char in chars:
        s = s.replace(char, replacement)
    return s

In [4]:
from sqlalchemy import create_engine,inspect
user = 'akshayparate'
passw = 'akshayparate'
host =  '182.48.201.235'
port = 3306
database = 'stock_fundamentals'
# Create an engine instance
engine = create_engine(f'mysql+pymysql://{user}:{passw}@{host}:{port}/{database}')

def generate_dataframes(annual,quarterly,stockName):
    annualdf = pd.DataFrame(annual).transpose()
    annualdf["Annual/Quarter"] = "Annual"
    quarter_df = pd.DataFrame(quarterly).transpose()
    quarter_df["Annual/Quarter"] = "Quarter"
    finalDf = pd.concat([annualdf, quarter_df], axis=0)
    finalDf["stockName"] = stockName
    finalDf = finalDf.reset_index()
    modified_columns_names = [change_column_names(s, characters_to_replace) for s in list(finalDf.columns)]
    finalDf.columns = modified_columns_names
    finalDf = finalDf.rename(columns={'index': 'Year_Date'})
    return finalDf

In [5]:
def check_if_data_already_exists_in_dataframe(existing_df,new_df,tableName):
    dataframes = [existing_df,new_df]
    all_columns = list(set().union(*(df.columns for df in dataframes)))
    # Ensure all DataFrames have the same columns
    for i, df in enumerate(dataframes):
        for col in all_columns:
            if col not in df.columns:
                df[col] = pd.NA
        dataframes[i] = df[all_columns]
    # Concatenate all DataFrames
    combined_df = pd.concat(dataframes, ignore_index=True)
    unique_stocks = combined_df['stockName'].unique()
    df_unique = pd.DataFrame()
    for u in unique_stocks:
        temp_df = combined_df[combined_df["stockName"]== u]
        temp_df = temp_df.drop_duplicates(subset=['Year_Date'])
        df_unique = pd.concat([df_unique,temp_df], ignore_index=True)
    return df_unique

In [6]:
def merge_dataframes(dataframes,existing_df,tableName,flag):
    all_columns = list(set().union(*(df.columns for df in dataframes)))
    # Ensure all DataFrames have the same columns
    for i, df in enumerate(dataframes):
        for col in all_columns:
            if col not in df.columns:
                df[col] = pd.NA
        dataframes[i] = df[all_columns]
    
    # Concatenate all DataFrames
    combined_df = pd.concat(dataframes, ignore_index=True)
    if flag:
        combined_df = check_if_data_already_exists_in_dataframe(existing_df,combined_df,tableName)
    return combined_df

In [7]:
def get_tables_from_database():
    inspector = inspect(engine)
    tables = inspector.get_table_names()
    return tables

In [8]:
def store_data(df,tableName,flag):
    if flag:
        df.to_sql(tableName, con=engine, if_exists='replace', index=False)
    else:
        df.to_sql(tableName, con=engine, if_exists='replace', index=False)
        

In [9]:
livemintDetails = pd.read_csv("LiveMint.csv")[["stockLink","stockName"]]
stockLinks = list(livemintDetails["stockLink"])
stockNames = list(livemintDetails["stockName"])

db_tables = get_tables_from_database()
flag = False

df_dict = {"cashFlowDf":[],"balanceSheetDf":[],"incomeSheetDf":[]}
for i in range(0,1863):
    print(i,stockNames[i])
    try:
        stockFinancialData = get_data(stockLinks[i])
        annualCashFlow,annualBalance,annualIncome,quarterlyCashFlow,quarterlyBalance,quarterlyIncome = data_structuring(stockFinancialData)
        cashFlowDf = generate_dataframes(annualCashFlow,quarterlyCashFlow,stockNames[i])
        balanceSheetDf = generate_dataframes(annualBalance,quarterlyBalance,stockNames[i])
        incomeSheetDf = generate_dataframes(annualIncome,quarterlyIncome,stockNames[i])
        df_dict["cashFlowDf"].append(cashFlowDf)
        df_dict["balanceSheetDf"].append(balanceSheetDf)
        df_dict["incomeSheetDf"].append(incomeSheetDf)
    except Exception as e:
        print(e)

if len(db_tables) != 0:
    flag = True
    existing_cashFlowDf = pd.read_sql('SELECT * FROM {}'.format("cashflow"), con=engine)
    existing_balanceSheetDf = pd.read_sql('SELECT * FROM {}'.format("balancesheet"), con=engine)
    existing_incomeSheetDf = pd.read_sql('SELECT * FROM {}'.format("incomesheet"), con=engine)
    cashFlowDf = merge_dataframes(df_dict["cashFlowDf"],existing_cashFlowDf,"cashflow",True)
    balanceSheetDf = merge_dataframes(df_dict["balanceSheetDf"],existing_balanceSheetDf,"balancesheet",True)
    incomeSheetDf = merge_dataframes(df_dict["incomeSheetDf"],existing_incomeSheetDf,"incomesheet",True)
else:
    flag = False
    cashFlowDf = merge_dataframes(df_dict["cashFlowDf"],[],"cashflow",False)
    balanceSheetDf = merge_dataframes(df_dict["balanceSheetDf"],[],"balancesheet",False)
    incomeSheetDf = merge_dataframes(df_dict["incomeSheetDf"],[],"incomesheet",False)

store_data(cashFlowDf,"cashflow",flag)
store_data(balanceSheetDf,"balancesheet",flag)
store_data(incomeSheetDf,"incomesheet",flag)


1500 Tijaria Polypipes
1501 Gujarat Lease Financing
1502 Integra Essentia
1503 Sands Power Switchgears
1504 Euro Ceramics
1505 Mic Electronics
1506 GACM Technologies
1507 Consolidated Construction Consortium
1508 Nu Tek India
Skipping this year for stock 'CAS'
1509 Hb Stockholdings
1510 Jumbo Bag
1511 Parenteral Drugs India
1512 Easun Reyrolle
1513 KSK Energy Ventures
1514 Premier
1515 Ortel Communications
1516 Impex Ferro Tech
1517 GB Global
1518 Orchid Pharma
1519 Ramgopal Polytex
object of type 'NoneType' has no len()
1520 Univa Foods
1521 Infomedia Press
1522 Diamond Power Infrastructure
1523 Salona Cotspin
Skipping this year for stock 'CAS'
Skipping this year for stock 'CAS'
1524 Visagar Polytex
1525 Shah Metacorp
1526 Rajdarshan Industries
1527 Next Mediaworks
1528 Nagreeka Capital &amp; Infra
1529 Landmark Property Development
1530 Creative Eye
1531 Ashapura Intimates Fashion
1532 Lakshmi Precision Screws
object of type 'NoneType' has no len()
1533 Eurotex Industries &amp; Expor

In [153]:



# Function to replace characters in a string


# Replace characters in each string in the list



In [107]:
livemintDetails.head()

,stockLink,stockName
0,https://www.livemint.com/market/market-stats/s...,State Bank Of India
1,https://www.livemint.com/market/market-stats/s...,ICICI Prudential Life Insurance Company
2,https://www.livemint.com/market/market-stats/s...,Reliance Industries
3,https://www.livemint.com/market/market-stats/s...,ICICI Bank
4,https://www.livemint.com/market/market-stats/s...,HDFC Bank


In [1]:
# # import requests 
# # from bs4 import BeautifulSoup 

# # liveMintStockDetails = {"stockLink":[],"stockName":[]}
# # for i in range(3016,5000):
# #     try:  
# #         URL = "https://www.livemint.com/market/market-stats/stocks-bajaj-finance-share-price-nse-bse-s000{}".format(i) 
# #         r = requests.get(URL) 
# #         soup = BeautifulSoup(r.content, 'html5lib')
# #         stockName_div = str(soup.find('div', attrs = {'class':'stockName'}).find('h1'))
# #         stockLink_div = str(soup.find('link', attrs = {'rel':'canonical'}))
# #         sub1 = "<h1>"
# #         sub2 = "</h1>"
# #         idx1 = stockName_div.find(sub1)
# #         idx2 = stockName_div.find(sub2)
# #         stockName = stockName_div[idx1 + len(sub1): idx2][:-12]
# #         print(i,stockName)
# #         liveMintStockDetails["stockName"].append(stockName)
# #         sub1 = '<link href="'
# #         sub2 = 'rel="canonical"/>'
# #         idx1 = stockLink_div.find(sub1)
# #         idx2 = stockLink_div.find(sub2)
# #         stockLink = stockLink_div[idx1 + len(sub1): idx2][:-2]
# #         liveMintStockDetails["stockLink"].append(stockLink)
# #     except Exception as e:
# #         print("Error")

# liveMintDf = pd.DataFrame(liveMintStockDetails)
# liveMintDf.to_csv("LiveMint.csv")